In [1]:
import librosa
y, sr = librosa.load("/raid/home/rajivratn/hemant_rajivratn/last/data/input.wav")
print(y.dtype)         # float32
print(y.min(), y.max())  # should be in [-1.0, 1.0]


float32
-0.67541337 0.7454596


In [ ]:
from audiotools import AudioSignal
signal = AudioSignal("/raid/home/rajivratn/hemant_rajivratn/last/data/input.wav")
signal.audio_data, signal.sample_rate

(tensor([[[0.0038, 0.0044, 0.0039,  ..., 0.0063, 0.0059, 0.0048]]]), 16000)

In [ ]:
y = signal.audio_data
print(y.dtype)         # float32
print(y.min(), y.max())  # should be in [-1.0, 1.0]

torch.float32
tensor(-0.6715) tensor(0.7465)


In [7]:
import soundfile as sf
waveform, sample_rate = sf.read("/raid/home/rajivratn/hemant_rajivratn/last/data/input.wav")
y = waveform
print(y.dtype)         # float32
print(y.min(), y.max())  # should be in [-1.0, 1.0]

float64
-0.671478271484375 0.7464599609375
